## 문서유사도 과제

In [ ]:
from konlpy.tag import Kkma
import re
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Twitter
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
def crawling(add):
    result =[]
    for p in range(50) :
        r = requests.get("{}{}" .format(add, p+1))
        soup = BeautifulSoup(r.content, "html.parser", from_encoding= 'ms949')
        title = soup.find_all("td", {"class":"title"})

        sub_result =[]

        for i in range(10) :
            sub_result.append(title[i].text.replace("\r", "").replace("\t", "").replace("\n", "").replace("신고", "").replace("군함도", "").replace("택시운전사", "").replace("청년경찰", "").replace("종의 전쟁", "").replace("혹성탈출", "").replace("노무현입니다", "").replace("500일의 썸머", "").replace("덩케르크", "").replace("애나벨", "").replace("이터널 선샤인", "").replace("스파이더맨", ""))
        result = result + sub_result

    return("".join(result))

In [ ]:
# 모든 리뷰들마다 영화제목이 크롤링됨.  이건 네티즌들이 쓴 것이 아니다. 그래새 싹 다 지워주도록 함수를 짰다.
# 그 외에도 신고 버튼에 있는 '신고' 글자나 \로 시작하는 이상한 문자들을 모두 지우도록 했다.
# 또 "".join() 함수를 이용해서 영화 하나에 해당하는 모든 후기들을 하나의 스트링으로 변환했다.

In [ ]:
taxi = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=after&page=")
gunhamdo = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146506&target=after&page=")
young_police = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=153652&target=after&page=")
war_of_species = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=143394&target=after&page=")
nomuhyun = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=162173&target=after&page=")
summer_500 = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=53152&target=after&page=")
dung = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146480&target=after&page=")
sunshine = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=38444&target=after&page=")
annabel = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=152341&target=after&page=")
spiderman = crawling("http://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=135874&target=after&page=")

In [ ]:
movies = [taxi, gunhamdo, young_police, war_of_species, nomuhyun, summer_500, dung, sunshine, annabel, spiderman]
# 한 영화에 대한 모든 리뷰들이 1개의 스트링으로 되어있다. movies는 총 10개의 스트링이 있는 리스트다.

In [ ]:
# 가장 많이 언급된 단어들을 확인해보자.
words_basket =[]

for movie in movies :
    words = twitter.pos(movie)
    for word in words :
        if word[1] in ["Noun", "Adjective"] and len(word[0]) >=2 :
            words_basket.append(word[0])

Counter(words_basket).most_common(30)

In [ ]:
# 쓸 데 없다고 판단되는 단어들은 지워주자.
movies = [movie.replace("영화", "") for movie in movies]
movies = [movie.replace("너무", "") for movie in movies]
movies = [movie.replace("정말", "") for movie in movies]
movies = [movie.replace("진짜", "") for movie in movies]
movies = [movie.replace("그냥", "") for movie in movies]
movies = [movie.replace("아니", "") for movie in movies]
movies = [movie.replace("있는", "") for movie in movies]
movies = [movie.replace("역시", "") for movie in movies]
movies = [movie.replace("있었", "") for movie in movies]
movies = [movie.replace("입니", "") for movie in movies]
movies = [movie.replace("커밍", "") for movie in movies]
movies = [movie.replace("썸머", "") for movie in movies]

In [ ]:
twitter = Twitter()
def word_separating(movies) :
    result=[]
    
    for movie in movies :      
        one_result = []

        words = twitter.pos(movie)
        for word in words:
            if word[1] in ["Noun", "Adjective"] and len(word[0]) >= 2 :
                one_result.append(word[0])
        one_result_a = " ".join(one_result)
        result.append(one_result_a)

    return(result)

word_list = word_separating(movies)

In [ ]:
# 영화 한 개에 대해 언급된 두 글자 이상의 명사, 형용사들을 모두 모아 1개의 스트링에 넣었다. 
# word_list는 영화가 10개니 총 10개의 스트링이 있는 상태다.

In [ ]:
count = CountVectorizer(min_df= 2)
tf_dtm = count.fit_transform(word_list).toarray()
tf_dtm = pd.DataFrame(tf_dtm, columns=count.get_feature_names(),
                     index= ['taxi', 'gunhamdo', 'young_police', 'war_of_species', 'nomuhyun', 'summer_500', 'dung','sunshine',
                              'annabel', 'spiderman'])

tf_dtm

In [ ]:
# 단어 빈도수에 따른 tf_dtm이다. 
# 하지만 문서 유사도를 구할 때는 단어들의 빈도수뿐만 아니라 단어들의 중요성까지 고려한 tfidf를 이용하는 것이 낫다. 구해보자.

In [ ]:
idf_maker = TfidfVectorizer(min_df=2)
tf_idf = idf_maker.fit_transform(word_list).toarray()
tf_idf_dtm = pd.DataFrame(tf_idf, columns= idf_maker.get_feature_names(), 
                      index= ['taxi', 'gunhamdo', 'young_police', 'war_of_species', 'nomuhyun', 'summer_500', 'dung','sunshine',
                              'annabel', 'spiderman'])

tf_idf_dtm

In [ ]:
# 이제 각 영화끼리 코사인 유사도를 구하자.
def cosin_similarity(doc1, doc2) :
    son = sum(doc1*doc2)
    mom = (sum((doc1)**2)*sum((doc2)**2))**0.5
    return(son/mom)

In [ ]:
x = np.zeros((10,10))
for j in range(10) :  
    for i in range(10) :
        x[j,i] = cosin_similarity(tf_idf_dtm.iloc[j], tf_idf_dtm.iloc[i].values)

In [ ]:
x = pd.DataFrame(x, index= ['taxi', 'gunhamdo', 'young_police', 'war_of_species', 'nomuhyun', 'summer_500', 'dung','sunshine',
                              'annabel', 'spiderman'],
                 columns = ['taxi', 'gunhamdo', 'young_police', 'war_of_species', 'nomuhyun', 'summer_500', 'dung','sunshine',
                              'annabel', 'spiderman'])

x

In [ ]:
# 택시운전사, 군함도 두 영화가 유사도가 매우 높다. 두 영화 모두 조국의 역사와 관련하여 감동적인 내용을 다루기  때문이라고 생각해볼 수 있다.
# 스파이더맨은 청년경찰, 혹성탈출과 유사도가 높게 나왔다. 세 영화가 모두 액션 영화이고 통쾌한 내용을 다루기 때문이라고 짐작할 수 있다.
# 이터널선샤인과 500일의 썸머는 모두 로맨스 코미디로 사랑에 대한 내용이다. 이것이 둘 간의 유사도가 높은 이유라고 생각해본다.
# 노무현입니다는 민주화에 대한 내용을 공통으로 하는 택시운전사와 가장 유사도가 높게 나왔다.
# 애나벨은 위 10개 중에서 홀로 공포영화 장르이다. 실제로도 모든 영화들과 유사도가 매우 낮게 나왔다.
# 흥미로웠던 것은 청년경찰, 혹성탈출이 택시운전사, 군함도와 비교적 높은 유사도를 보였다는 것이다.  내용과 분위기가 아예 다른데 의외의 결과였다. 공통점을 찾아보니 네 영화 모두 관객이 300만 이상이거나 예매 순위가 3위 안에 드는 소위 '재미있는 영화''였음을 알 수 있었다. 아마 "재미"나 "최고"라는 단어들이 네 영화를 묶어낸 것이 아닌가 추측해본다. 덩케르크가 다른 영화들에 비해 이 네 영화들과 비교적 높고 비슷한 유사도를 보이는 것도 비슷한 이유리라 생각한다. 조사 결과, 덩케르크도 예매율이 2위, 관객수 약 3백만에 이를만큼 재밌고 인기있는 영화였다.

In [ ]:
# 위 분석에 따르면 영화가 다루는 소재와 내용,  영화의 장르, 흥행도에 따라서 유사도가 결정됨을 알 수 있다.